# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# create path for output data from WeatherPY
# print dataframe
Vacation_pd = pd.read_csv("Resources/Weather_Output.csv")
Vacation_pd


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# configure api key
# create variable for laocations
# create variable for humidity 
gmaps.configure(api_key=g_key)
locations = Vacation_pd[["Lat", "Lng"]]
Humidity = Vacation_pd["Humidity"].astype(float)

In [ ]:
# configure heat layer map 
# print heat layer map 
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# set max temp between 75 and 80 degrees
# drop nulls
#drint dataframe
Vacation_conditional_df = Vacation_pd.loc[(Vacation_pd["Max Temp"] >=75) & (Vacation_pd["Max Temp"] <=80) ]
Vacation_clean_df=Vacation_conditional_df.dropna()
Vacation_clean_df


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# place celan dataframe into hotel dataframe 
# create hotel column
# print dataframe 
hotel_df = Vacation_clean_df
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Check the latitudes and longitudes 
Hotel_location = Vacation_clean_df[["Lat", "Lng"]]
Hotel_location

In [ ]:
# create params for radius, location type, and key
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
#create for loop to find hotel locations 
# create try except to skip locations with KeyError and IndexError
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}" 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")

In [ ]:
#print new dataframe with hotel names 
hotel_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

Hotel_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(Hotel_marker)


# Display figure
fig